# Is It A
This notebook will take in the first 5 characters of the Japanese Hiragana alphabet and the train a model to disguish between these characters to train a character recognition model.
In this case, the model from SageMaker will be deployed to AWS DeepLens device.

--- 

1. [Setup](#Setup)

## Setup

### Permissions and Environment Variables

In [ ]:
import os
import urllib.request
import boto3, botocore
import json


import sagemaker
from sagemaker import get_execution_role

import mxnet as mx
mxnet_path = mx.__file__[ : mx.__file__.rfind('/')]
print(mxnet_path)

role = get_execution_role()
print(role)

sess = sagemaker.Session()

### S3 Bucket

In [ ]:
# Replace the BUCKET name to your bucket
BUCKET = 'deeplens-isit'
PREFIX = 'isita'

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri

# training_image = sagemaker.image_uris.retrieve(framework='mxnet',
#                                                region=sess.boto_region_name,
#                                                version=mx.__version__,
#                                                py_version='py37',
#                                                instance_type='ml.t2.medium',
#                                                image_scope='training')

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

print (training_image)

### S3 Bucket Permissions Check

In [ ]:
test_data = 'TestData'
s3 = boto3.resource('s3')
object = s3.Object(BUCKET, PREFIX+"/test.txt")
try:
    object.put(Body=test_data)
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "AccessDenied":
        #cannot write on the bucket
        print("Bucket "+BUCKET+"is not writeable, make sure you have the right permissions")
    else:
        if e.response['Error']['Code'] == "NoSuchBucket":
            #Bucket does not exist
            print("Bucket"+BUCKET+" does not exist")
        else:
            raise
else:
    print("Bucket access is Ok")
    object.delete()

In [ ]:
## Build The Character Dataset

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_images(item_name, images_to_show=-1):
    _im_list = !ls $IMAGES_DIR/$item_name

    NUM_COLS = 3
    if images_to_show == -1:
        IM_COUNT = len(_im_list)
    else:
        IM_COUNT = images_to_show
    
    print('Displaying images category ' + item_name + ' count: ' + str(IM_COUNT) + ' images.')
    
    NUM_ROWS = int(IM_COUNT / NUM_COLS)
    if ((IM_COUNT % NUM_COLS) > 0):
        NUM_ROWS += 1

    fig, axarr = plt.subplots(NUM_ROWS, NUM_COLS)
    fig.set_size_inches(10.0, 10.0, forward=True)

    curr_row = 0
    for curr_img in range(IM_COUNT):
        # fetch the url as a file type object, then read the image
        f = IMAGES_DIR + item_name + '/' + _im_list[curr_img]
        a = plt.imread(f)

        # find the column by taking the current index modulo 3
        col = curr_img % NUM_ROWS
        # plot on relevant subplot
        if NUM_ROWS == 1:
            axarr[curr_row].imshow(a)
        else:
            axarr[col, curr_row].imshow(a)
        if col == (NUM_ROWS - 1):
            # we have finished the current row, so increment row counter
            curr_row += 1

    fig.tight_layout()
    plt.show()
        
    # Clean up
    plt.clf()
    plt.cla()
    plt.close()

In [ ]:
IMAGES_DIR = '../training/images/'
show_images("A", images_to_show=3)
show_images("I", images_to_show=3)
show_images("U", images_to_show=3)
show_images("E", images_to_show=3)
show_images("O", images_to_show=3)

### Copy Images to S3

In [ ]:
DEST_BUCKET = 's3://'+BUCKET+'/'+PREFIX+'/images/'

!aws s3 cp --recursive $IMAGES_DIR $DEST_BUCKET --quiet

In [ ]:
# Ensure that the newly created directories containing the downloaded data are structured as expected
!aws s3 ls $DEST_BUCKET

#### Prepare "list" files with train-val split

The image classification algorithm can take two types of input formats. The first is a [RecordIO format](https://mxnet.apache.org/api/faq/recordio) (content type: application/x-recordio) and the other is a Image list format (.lst file). These file formats allows for efficient loading of images when training the model. In this example we will be using the Image list format (.lst file). A .lst file is a tab-separated file with three columns that contains a list of image files. The first column specifies the image index, the second column specifies the class label index for the image, and the third column specifies the relative path of the image file. The RecordIO file contains the actual pixel data for the images.

To be able to create the .rec files, we first need to split the data into training and validation sets (after shuffling) and create two list files for each. Here our split into train, validation and test (specified by the `0.7` parameter below for test). We keep 0.02% to test the model.

The image and lst files will be converted to RecordIO file internally by the image classification algorithm. But if you want do the conversion, the following cell shows how to do it using the [im2rec](https://github.com/apache/incubator-mxnet/blob/master/tools/im2rec.py) tool. Note that this is just an example of creating RecordIO files. We are **_not_** using them for training in this notebook.

In [ ]:
!python $mxnet_path/tools/im2rec.py --list --recursive --test-ratio=0.02 --train-ratio 0.7 characters $IMAGES_DIR

## Data Preperation
### Save lst files to S3
Training models is easy with Amazon SageMaker. When you’re ready to train in SageMaker, simply specify the location of your data in Amazon S3, and indicate the type and quantity of SageMaker ML instances you need. SageMaker sets up a distributed compute cluster, performs the training, outputs the result to Amazon S3, and tears down the cluster when complete. 
To use Amazon Sagemaker training we must first transfer our input data to Amazon S3.

In [ ]:
s3train_lst = 's3://{}/{}/train_lst/'.format(BUCKET, PREFIX)
s3validation_lst = 's3://{}/{}/validation_lst/'.format(BUCKET, PREFIX)

# upload the lst files to train_lst and validation_lst channels
!aws s3 cp characters_train.lst $s3train_lst --quiet
!aws s3 cp characters_val.lst $s3validation_lst --quiet

#### Retrieve dataset size
Let's see the size of train, validation and test datasets

In [ ]:
f = open('characters_train.lst', 'r')
train_samples = sum(1 for line in f)
f.close()
f = open('characters_val.lst', 'r')
val_samples = sum(1 for line in f)
f.close()
f = open('characters_test.lst', 'r')
test_samples = sum(1 for line in f)
f.close()
print('train_samples:', train_samples)
print('val_samples:', val_samples)
print('test_samples:', test_samples)

This marks the end of the data preparation phase.

## Training Model Preperation
Training a good model from scratch can take a long time. Fortunately, we're able to use transfer learning to fine-tune a model that has been trained on millions of images. Transfer learning allows us to train a model to recognize new classes in minutes instead of hours or days that it would normally take to train the model from scratch. Transfer learning requires a lot less data to train a model than from scratch (hundreds instead of tens of thousands).

There are hyperparameters that are specific to the algorithm. These are:

* **num_layers**: The number of layers (depth) for the network. We use 18 in this samples but other values such as 50, 152 can be used.
* **use_pretrained_model**: Set to 1 to use pretrained model for transfer learning.
* **image_shape**: The input image dimensions,'num_channels, height, width', for the network. It should be no larger than the actual image size. The number of channels should be same as the actual image.
* **num_classes**: This is the number of output classes for the new dataset. For us, we have 
* **num_training_samples**: This is the total number of training samples. It is set to 15240 for caltech dataset with the current split.
* **mini_batch_size**: The number of training samples used for each mini batch. In distributed training, the number of training samples used per batch will be N * mini_batch_size where N is the number of hosts on which training is run.
* **epochs**: Number of training epochs.
* **learning_rate**: Learning rate for training.
* **top_k**: Report the top-k accuracy during training.
* **resize**: Resize the image before using it for training. The images are resized so that the shortest side is of this parameter. If the parameter is not set, then the training data is used as such without resizing.
* **precision_dtype**: Training datatype precision (default: float32). If set to 'float16', the training will be done in mixed_precision mode and will be faster than float32 mode


In [ ]:
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters({
    "num_layers": 1,
    "use_pretrained_model": 1,
    "image_shape": "3,64,63",
    "num_classes": 3,
    "mini_batch_size": 128,
    "epochs": 10,
    "learning_rate": 0.01,
    "top_k": 2,
    "num_training_samples": train_samples,
    "resize": 64})

### Fine-tuning the Image Classification Model
Now that we are done with all the setup that is needed, we are ready to train our trash detector. To begin, let us create a ``sageMaker.estimator.Estimator`` object. This estimator will launch the training job.
### Training parameters
There are two kinds of parameters that need to be set for training. The first one are the parameters for the training job. These include:

* **Training instance count**: This is the number of instances on which to run the training. When the number of instances is greater than one, then the image classification algorithm will run in distributed settings. 
* **Training instance type**: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training 
* **Output path**: This the s3 folder in which the training output is stored

In [ ]:
s3_output_location = 's3://{}/{}/output'.format(BUCKET, PREFIX)
ic = sagemaker.estimator.Estimator(training_image,
                                         role,
                                         instance_count=1, 
                                         instance_type='ml.p2.xlarge',
                                         volume_size = 50,
                                         max_run = 360000,
                                         input_mode= 'File',
                                         output_path=s3_output_location,
                                         sagemaker_session=sess,
                                         base_job_name='ic-characters',
                                         hyperparameters=hyperparameters)

### Input data specification
Set the data type and channels used for training

In [ ]:
s3images = 's3://{}/{}/images/'.format(BUCKET, PREFIX)

train_data = sagemaker.inputs.TrainingInput(s3images, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data = sagemaker.inputs.TrainingInput(s3images, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')
train_data_lst = sagemaker.inputs.TrainingInput(s3train_lst, distribution='FullyReplicated', 
                        content_type='application/x-image', s3_data_type='S3Prefix')
validation_data_lst = sagemaker.inputs.TrainingInput(s3validation_lst, distribution='FullyReplicated', 
                             content_type='application/x-image', s3_data_type='S3Prefix')

data_channels = {'train': train_data, 'validation': validation_data, 
                 'train_lst': train_data_lst, 'validation_lst': validation_data_lst}

## Train
Start training by calling the fit method in the estimator. This command will take a while (<10m) to complete.

In [ ]:
ic.fit(inputs=data_channels, logs=True)

The output from the above command will have the model accuracy and the time it took to run the training. 

*You can also view these details by navigating to ``Training -> Training Jobs -> job_name -> View logs`` in the Amazon SageMaker console*

### Review Trained Model Output
The model trained above can now be found in the `s3://<YOUR_BUCKET>/<PREFIX>/output` path.

MODEL_PATH = ic.model_data
print(MODEL_PATH)

## Deploy to a Sagemaker endpoint
After training your model is complete, you can test your model by asking it to predict the class of a sample trash image that the model has not seen before. This step is called inference.

Amazon SageMaker provides an HTTPS endpoint where your machine learning model is available to provide inferences. For more information see the [Amazon SageMaker documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
ic_infer = ic.deploy(initiainstance_count=1, instance_type='ml.t2.medium')

### Test the images against the endpoint
We will use the test images that were kept aside for testing.

In [ ]:
from IPython.display import Image, display
import json
import numpy as np

object_categories = ['A', 'I', 'U', 'E', 'O']

def test_model():
    preds = []
    acts  = []
    num_errors = 0
    with open('characters_test.lst', 'r') as f:
        for line in f:
            stripped_line = str(line.strip()).split("\t")
            file_path = stripped_line[2]
            category = int(float(stripped_line[1]))
            with open(IMAGES_DIR + stripped_line[2], 'rb') as f:
                payload = f.read()
                payload = bytearray(payload)

                result = json.loads(ic_infer.predict(payload, initial_args={'ContentType': 'application/x-image'}))
            # the result will output the probabilities for all classes
            # find the class with maximum probability and print the class index
            index = np.argmax(result)
            act = object_categories[category]
            pred = object_categories[index]
            conf = result[index]
            print("Result: Predicted: {}, Confidence: {:.2f}, Actual: {} ".format(pred, conf, act))
            acts.append(category)
            preds.append(index)
            if (pred != act):
                num_errors += 1
                print('ERROR on image -- Predicted: {}, Confidence: {:.2f}, Actual: {}'.format(pred, conf, act))
            display(Image(filename=IMAGES_DIR + stripped_line[2], width=64, height=63))

    return num_errors, preds, acts
num_errors, preds, acts = test_model()

## Display confusion matrix showing 'true' and 'predicted' labels

A confusion matrix is a table that is often used to describe the performance of a classification model (or "classifier") on a set of test data for which the true values are known. It's a table  with two dimensions ("actual" and "predicted"), and identical sets of "classes" in both dimensions (each combination of dimension and class is a variable in the contingency table). The diagonal values in the table indicate a match between the predicted class and the actual class. 

For more details go to [Confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix) (Wikipedia)

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools
COLOR = 'green'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR
def plot_confusion_matrix(cm, classes,
                          class_name_list,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.GnBu):
    plt.figure(figsize=(7,7))
    plt.grid(False)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), 
                                  range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.gca().set_xticklabels(class_name_list)
    plt.gca().set_yticklabels(class_name_list)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def create_and_plot_confusion_matrix(actual, predicted):
    cnf_matrix = confusion_matrix(actual, np.asarray(predicted),labels=range(len(object_categories)))
    plot_confusion_matrix(cnf_matrix, classes=range(len(object_categories)), class_name_list=object_categories)


create_and_plot_confusion_matrix(acts, preds)

## (Optional) Clean-Up
If you're ready to be done with this notebook, please run the cell below. This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.
*NOTE* : To save on costs, stop your notebook instances and delete the model edpoint when not in use

In [ ]:
sess.delete_endpoint(ic_infer.endpoint)
print("Completed")

## Rename model to deploy to AWS DeepLens
The MxNet model that is stored in the S3 bucket contains 2 files: the params file and a symbol.json file. To simplify deployment to AWS DeepLens, we'll modify the params file so that you do not need to specify the number of epochs the model was trained for.

In [ ]:
import glob

!rm -rf data/$PREFIX/tmp && mkdir -p data/$PREFIX/tmp
!aws s3 cp $MODEL_PATH data/$PREFIX/tmp
!tar -xzvf data/$PREFIX/tmp/model.tar.gz -C data/$PREFIX/tmp

params_file_name = glob.glob('./data/' + PREFIX + '/tmp/*.params')[0]

In [ ]:
!mv $params_file_name data/$PREFIX/tmp/image-classification-0000.params

In [ ]:
!tar -cvzf ./model.tar.gz -C data/$PREFIX/tmp ./image-classification-0000.params ./image-classification-symbol.json

In [ ]:
!aws s3 cp model.tar.gz $MODEL_PATH

## Next Steps

At this point, you have completed:
* Training a model with Amazon Sagemaker using transfer learning

Next you'll deploy this model to AWS DeepLens. If you have started this notebook as part of a tutorial, please go back to the next step in the tutorial. If you have found this notebook through other channels, please go to [awsdeeplens.recipes](http://awsdeeplens.recipes) and select the Trash Detector tutorial to continue.